In [3]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier nltk


from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Loading the NLTK-Ressources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [4]:
# Get the describtion of the POS Tags
def get_wordnet_pos(treebank_tag):
    """Konvertiert POS-Tag in ein Format, das vom WordNet-Lemmatizer unterstützt wird."""
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [5]:
# Lemmatize a text
def lemmatize_text(text):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in pos_tags]
    return ' '.join(lemmatized_tokens)

In [6]:
# Definition der Funktion zur Vorverarbeitung von Dokumenten
def preprocess_documents(documents):
    """Anwendet Lemmatization auf alle Dokumente."""
    for doc in documents:
        doc['text'] = lemmatize_text(doc['text'])
        yield doc

In [7]:
# Definition der Funktion zum Erstellen eines Index
def create_index(documents, stopwords):
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stopwords=stopwords)
    index_ref = indexer.index(preprocess_documents(documents))
    return pt.IndexFactory.of(index_ref)

In [8]:
# Definition der Funktion zum Lesen einer Textdatei und Konvertieren in ein Array
def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Entfernen von Zeilenumbrüchen und Konvertierung in Strings
            array = [line.strip() for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

In [9]:
# Lesen der Stopwords-Datei und Konvertieren in ein Array
file_path = "../terrier-stopwordslist.txt"
stopwords = read_text_file_to_array(file_path)

# Erstellen des benutzerdefinierten Index mit Lemmatization
costum_index = create_index(pt_dataset.get_corpus_iter(), stopwords)

# Erstellen der BatchRetrieve-Instanzen
XSqrA_M = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_costum = pt.BatchRetrieve(costum_index, wmodel="XSqrA_M")

File ../terrier-stopwordslist.txt not found.
Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-inputs.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 39.4M/39.4M [00:00<00:00, 76.0MiB/s]


Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/


ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [08:49<00:00, 239.95it/s] 


In [10]:
# Evaluation der Modelle
pt.Experiment(
    [XSqrA_M, XSqrA_M_costum],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names=["XSqrA_M", "XSqrA_M_costum"]
)

Download from the Incubator: https://files.webis.de/data-in-production/data-research/tira-zenodo-dump-preparation/ir-lab-sose2024/ir-acl-anthology-20240504-truth.zip?download=1
	This is only used for last spot checks before archival to Zenodo.


Download: 100%|██████████| 29.6k/29.6k [00:00<00:00, 1.59MiB/s]

Download finished. Extract...
Extraction finished:  /root/.tira/extracted_datasets/ir-lab-sose-2024/ir-acl-anthology-20240504-training/
There are multiple query fields available: ('text', 'title', 'query', 'description', 'narrative'). To use with pyterrier, provide variant or modify dataframe to add query column.


,name,ndcg_cut.10,recip_rank,recall_100,P_10
0,XSqrA_M,0.448211,0.650289,0.592214,0.400000
1,XSqrA_M_costum,0.411850,0.653066,0.582741,0.372059
